In [1]:
import pandas as pd
import numpy as np
import cv2
import os

In [3]:
labels=[i[:3] for i in os.listdir("train")]

In [4]:
def load_images(folder_path):
    images=[]
    for filename in os.listdir(folder_path):
        img=cv2.imread(os.path.join(folder_path,filename))
        if img is not None:
            image=cv2.resize(img,(64,64))
            images.append(image)
    return images

In [5]:
images=load_images("train")

In [6]:
from skimage.feature import hog
def image_features(images):
    features=[]
    for image in images:
        gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        hog_features=hog(gray_image,block_norm='L2-Hys',pixels_per_cell=(8,8),cells_per_block=(2,2))
        features.append(hog_features)
    return np.array(features)
hog_features=image_features(images)

In [7]:
df=pd.DataFrame(hog_features)
df['target']=labels

In [8]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['target']=encoder.fit_transform(df['target'])

In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1755,1756,1757,1758,1759,1760,1761,1762,1763,target
0,0.305868,0.064261,0.010817,0.000000,0.059005,0.141722,0.110887,0.169559,0.305868,0.305868,...,0.046604,0.022060,0.051672,0.116838,0.089043,0.098032,0.023665,0.035107,0.049882,0
1,0.269501,0.039625,0.037222,0.223956,0.154846,0.036283,0.049265,0.022506,0.075312,0.265965,...,0.236933,0.190674,0.178775,0.236933,0.188337,0.062029,0.035534,0.082725,0.090604,0
2,0.075582,0.063367,0.133397,0.230637,0.305685,0.137356,0.028371,0.025553,0.024630,0.032594,...,0.265868,0.122746,0.128171,0.117027,0.084787,0.265868,0.139590,0.143814,0.157300,0
3,0.122261,0.011962,0.037389,0.072189,0.070585,0.041724,0.074590,0.040957,0.025248,0.145032,...,0.309203,0.038008,0.042686,0.000000,0.003606,0.009176,0.000000,0.157363,0.309203,0
4,0.188206,0.056430,0.159058,0.215132,0.214091,0.242554,0.242554,0.119614,0.048317,0.242554,...,0.205116,0.101634,0.189592,0.070024,0.188827,0.063668,0.086440,0.121408,0.090100,0


In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop('target',axis=1),df.target,stratify=df.target,random_state=43,test_size=0.3)

In [13]:
from sklearn.svm import SVC
model=SVC()
model.fit(x_train,y_train)

In [12]:
from sklearn.metrics import classification_report,confusion_matrix
y_pred=model.predict(x_test)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.77      0.78      0.77      3685
           1       0.78      0.77      0.78      3815

    accuracy                           0.77      7500
   macro avg       0.77      0.77      0.77      7500
weighted avg       0.77      0.77      0.77      7500

